In [2]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [7]:
!pip3 install stable-baselines3

In [6]:
!pip3 install mss pydirectinput pytesseract

In [5]:
!pip3 install opencv-python

In [4]:
#chrome://dino/

In [172]:
from mss import mss
import pydirectinput
import cv2
import numpy as np
import pytesseract
from matplotlib import pyplot as plt
import time
from gym import Env
from gym.spaces import Box, Discrete

In [171]:
class DinoGame(Env):

    def __init__(self):
        super().__init__()

        # Setup spaces
        self.observation_space = Box(low=0, high=255, shape=(1,83,100), dtype=np.uint8)
        self.action_space = Discrete(3)

        # Capture game frames
        self.cap = mss()
        self.game_location = {'top': 300, 'left': 0, 'width': 600, 'height': 500}
        self.done_location = {'top': 405, 'left': 630, 'width': 660, 'height': 70}

    # What is called to do something in the game
    def step(self, action):
        # Action key - 0 = Space, 1 = Duck(down), 2 = No action (no op)
        def step(self, action):
            action_map = {
                0:'space',
                1: 'down', 
                2: 'no_op'
            }
            if action !=2:
                pydirectinput.press(action_map[action])
            done, done_cap = self.get_done()
            observation = self.get_observation()
            reward = 1 
            info = {}
            return observation, reward, done, info

    # Restart the game
    def reset(self):
        time.sleep(1)
        pydirectinput.click(x=150, y=150)
        pydirectinput.press('space')
        return self.get_observation()

    # Visualize the game
    def render(self):
        cv2.imshow('Game', self.current_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            self.close()

    # This closes down the observation
    def close(self):
        cv2.destroyAllWindows()

    # Get the part of the observation of the game that we want
    def get_observation(self):
        raw = np.array(self.cap.grab(self.game_location))[:,:,:3].astype(np.uint8)
        gray = cv2.cvtColor(raw, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (100,83))
        channel = np.reshape(resized, (1,83,100))
        return channel

    # Get the done text
    def get_done(self):
        done_cap = np.array(self.cap.grab(self.done_location))
        done_strings = ['GAME', 'GAHE']
        done=False
        done = False
        res = pytesseract.image_to_string(done_cap)[:4]
        if res in done_strings:
            done = True
        return done, done_cap